In [1]:
import helpers
from helpers import write_result, load_malawi_data, split_data, SAVE_PATH

from pathlib import Path
import importlib

import pandas as pd
import numpy as np

from multiprocessing import Pool

path_to_feature_sets = Path('/home/selker/eop/eop/select_predictors/2019')

import copy
import csv
from pathlib import Path
from functools import partial
from multiprocessing import Pool
import os

import numpy as np
import pandas as pd
from opt_targeted_transfers import (
    ConditionalTargetedTransfers,
    HybridTargetedTransfers,
    UnconditionalTargetedTransfers,
    GapTargetedTransfers
)

from helpers import write_result, load_malawi_data, split_data, SAVE_PATH


def run_simulation(
    country="malawi",
    constraint="unconditional",
    method="qr",
    condtol=None,  # iterable
    uncondtol=None,  # iterable
    save="malawi_results.csv",
    features=None,
    extra_run_labels=None,
    district=None
):

    save.mkdir(exist_ok=True)

    d = len(features)
    
    if extra_run_labels is None:
        extra_run_labels = dict()

    if features is None:
        # TODO: Redo this signature
        raise ValueError('Specify some features!')

    # A semi-hack to replace the incorrect d that was populated before.
    extra_run_labels.update({'d': d})
    
    X, y, r, _ = load_malawi_data(features, district)

    
    (X_train, y_train, r_train), (X_test, y_test, r_test) = split_data(
        X=X, y=y, r=r, p=0.6
    )

    base_name = f'{country}_d={d}_{constraint}'

    if constraint == "unconditional":

        tt = UnconditionalTargetedTransfers(c_bar=2.15)
        
        tt.fit(X_train, y_train, r_train)
        for tol in uncondtol:
            tt.set_unconditional_tolerance(tol)
            tt.run_opt(
                    X_test,
                    r_test,
                    path=save / f'{base_name}_uncondtol={tol}_opt.csv'
                )
            res = tt.evaluate(X_test, y_test, r_test)
            write_result(save / "{}.csv".format(country), res, extra_run_labels)
            tt.evaluate_equity(
                X_test,
                y_test,
                path=save / f'equity_{base_name}_uncondtol={tol}_opt.csv'
            )

    elif constraint == "conditional":
        tt = ConditionalTargetedTransfers(method=method, c_bar=2.15)

        if method == "density":
            tt.fit(X_train, y_train, r_train)
            for tol in condtol:
                tt.set_conditional_tolerance(tol)
                tt.run_opt(
                        X_test,
                        r_test,
                    )
                res = tt.evaluate(X_test, y_test, r_test)
                print('saving')
                write_result(save / f'{country}.csv', res, extra_run_labels)
                tt.evaluate_equity(
                    X_test,
                    y_test,
                    path=save / f'equity_{base_name}_undcondtol={tol}_condtol={tol}'
                )
        elif method == "qr":
            for tol in condtol:
                tt.set_conditional_tolerance(tol)
                tt.fit(X_train, y_train, r_train)
                tt.run_opt(
                        X_test,
                        r_test,
                    )
                res = tt.evaluate(X_test, y_test, r_test)
                write_result(save / f'{country}.csv', res, extra_run_labels)
                tt.evaluate_equity(
                    X_test,
                    y_test,
                    path=save / f'equity_{base_name}_uncondtol={tol}_condtol={tol}.csv'
                )

    elif constraint == "hybrid":

        tt = HybridTargetedTransfers(c_bar=2.15)
        tt.fit(X_train, y_train, r_train)
        
        for tol1 in uncondtol:
            for tol2 in condtol:
                tt.set_conditional_tolerance(tol2)
                tt.set_unconditional_tolerance(tol1)
                tt.run_opt(
                    X_test,
                    r_test,
                    path=save / f'{base_name}_uncondtol={tol1}_condtol={tol2}_opt.csv'
                )
                res = tt.evaluate(X_test, y_test, r_test)
                write_result(save / "{}.csv".format(country), res, extra_run_labels)
                tt.evaluate_equity(
                    X_test,
                    y_test,
                    path=save / f'equity_{base_name}_uncondtol={tol1}_condtol={tol2}.csv'
                )


def run_gap_targeting_grid(
    country="malawi",
    method="qr",
    save="malawi_results.csv",
    features=None,
    extra_run_labels=None,
    district=None,
    ns_quantiles=[20],
    hidden_layer_sizes=[64],
    lambdas=None  # iterable
):

    save.mkdir(exist_ok=True)

    d = len(features)
    
    if extra_run_labels is None:
        extra_run_labels = dict()

    if features is None:
        # TODO: Redo this signature
        raise ValueError('Specify some features!')

    # A semi-hack to replace the incorrect d that was populated before.
    extra_run_labels.update({'d': d})
    
    X, y, r, _ = load_malawi_data(features, district)

    (X_train, y_train, r_train), (X_test, y_test, r_test) = split_data(
        X=X, y=y, r=r, p=0.6
    )

    base_name = f'{country}_d={d}_gap'

    if lambdas is None:
        raise ValueError('need lambdas')

    tt = GapTargetedTransfers(c_bar=2.15)

    for n_quantiles in ns_quantiles:
        
        for hidden_layer_size in hidden_layer_sizes:

            tt.fit(X_train, y_train, r_train, n_quantiles=n_quantiles, hidden_layer_size=hidden_layer_size)
        
            for lambda_ in lambdas:
            
                tt.run_opt(
                    X_test,
                    lambda_
                )
                res = tt.evaluate(X_test, y_test, r_test)
                write_result(
                    save / "{}.csv".format(country), 
                    res,
                    {
                        **extra_run_labels, 
                        'lambda': lambda_,
                        'n_quantiles': n_quantiles,
                        'hidden_layer_size': hidden_layer_size
                    }
                )

def malawi_runs(
    features, 
    district=None, 
    extra_run_labels=None,  # dictionary
    ds=None,  # number(s) of features to keep
    multiprocess=True
):

    tolerances = [0.2]

    constraints = ["unconditional",]
    quantile_methods = ["qr", "density"]

    args = []

    if ds is None:
        ds=[len(features)]

    for d in ds:
        features_d = features[:d]
        for constraint in constraints:
            if constraint == "conditional":
                for quantile_method in quantile_methods:

                    args.append(
                        {
                            'constraint': constraint, 
                            'features': features_d,
                            'method': quantile_method, 
                            'condtol': tolerances,
                        }
                    )

            elif constraint == "unconditional":

                args.append(
                        {
                            'constraint': constraint,  
                            'features': features_d,
                            'uncondtol': tolerances,
                        }
                )

            elif constraint == "hybrid":
                args.append(
                        {
                            'constraint': constraint,  
                            'features': features_d,
                            'uncondtol': tolerances,
                            'condtol': tolerances
                        }
                )
    for arg in args:
        arg.update(
            {
                'country': 'malawi', 
                'extra_run_labels': extra_run_labels,
                'district': district
            }
        )

    if multiprocess:

        pool = Pool()
    
        pool.map_async(
            _unpack_and_run,
            args,
            error_callback=error_callback
        )
    
        pool.close()
        pool.join()

    else:

        for args_dictionary in args:
            _unpack_and_run(args_dictionary)


def _unpack_and_run(args_dictionary):
    return run_simulation(
        save=SAVE_PATH / 'test',
        **args_dictionary
    )

def error_callback(error):
    print(f"Process failed: {str(error)}")

## Run simulations

### Subnational analysis

In [7]:
%%time

for covariates_file in list((path_to_feature_sets / 'district_level_covariates').iterdir()):
    covariates = pd.read_csv(covariates_file)
    district = covariates_file.stem
    malawi_runs(
        covariates.covariate.values,
        district,
        extra_run_labels={
            'features': f'{district}',
            'with_weights': True
        }
    )

Mzimba
KNOTS:[-2.34803814 -2.34803814 -2.34803814 -2.34803814 -1.28341613 -0.82362213
 -0.24707157  0.27167291  2.81226406  2.81226406  2.81226406  2.81226406]
Fitting conditional densities vs glm spline method...


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████| 300/300 [00:00<00:00, 317.42it/s, loss=-0.164, val_loss=0.306]


Final Theta: tensor([[-0.1544,  0.5526, -0.1375,  0.4983, -0.2465, -0.3276, -0.0972,  0.6805,
         -0.6458,  0.0861, -0.4419, -0.5094, -0.1841,  0.7198,  0.6408, -0.5507,
         -0.3028,  0.5536,  0.7987, -0.3572,  0.5952,  0.0801],
        [-0.0656, -0.2831, -0.2315, -0.4622,  0.1809, -0.5093, -0.3191,  0.9695,
          0.6269, -0.9245,  0.5573,  0.0287,  0.1110, -0.3129, -0.6582,  0.0303,
          0.0381, -0.4508,  0.3872,  1.0754, -0.3550,  0.6837],
        [-0.1132, -0.0102, -0.1941, -0.2999,  0.1919, -0.6003, -0.8005,  0.4692,
         -0.8482, -0.0372, -1.1334, -0.1912,  1.0697,  0.7947,  0.4652,  0.6582,
         -0.0235, -0.0570, -0.1952, -0.2520,  0.5169, -0.2046],
        [ 0.8026, -0.2430, -0.5440, -0.3091,  0.7252,  0.3661, -0.8286,  0.1322,
         -0.5938, -0.0451,  0.1341, -0.6575, -0.3621,  0.2681,  0.2388,  0.1103,
          0.0234,  0.6200, -0.3180,  0.2791,  0.5796,  0.8780],
        [ 0.0479,  0.2881, -0.0842,  0.4557,  0.6331, -0.9113, -0.1803, -0.9402,
  

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 200/200 [00:03<00:00, 63.49it/s]


CPU times: user 33.9 s, sys: 16.3 s, total: 50.3 s
Wall time: 9.66 s


In [11]:
full_country_covariates = pd.read_csv(path_to_feature_sets / 'covariates_below_5_durable.csv').covariate.values
# full_country_covariates = pd.read_csv(path_to_feature_sets / 'covariates_country_all.csv').covariate.values

malawi_runs(
    full_country_covariates,
    multiprocess=False,
    extra_run_labels = {
        'features': 'full_country',
        'with_weights': True
    }
)

Fitting conditional densities vs glm spline method...


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████| 300/300 [00:05<00:00, 57.03it/s, loss=-0.183, val_loss=-0.103]


Final Theta: tensor([[ 0.0503,  0.9510, -0.3619,  ..., -0.1170, -0.4513, -0.0466],
        [ 0.1628, -0.1270,  0.4220,  ..., -0.9314,  0.7283,  0.3475],
        [ 1.0777, -0.2233, -0.5612,  ..., -0.1165, -0.2739,  0.4028],
        ...,
        [-0.4731, -0.2828, -0.2138,  ...,  0.2925,  0.3721,  0.5432],
        [-0.0688,  0.0512,  0.8323,  ...,  0.8371, -0.3488,  0.2082],
        [-0.6935,  0.1320, -0.7672,  ..., -0.5061,  0.3029, -0.3506]],
       dtype=torch.float64)
Alpha range: nan, nan


100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 200/200 [00:47<00:00,  4.19it/s]


## Gap targeting

In [2]:
selected_durable_verifiable_covariates = pd.read_csv(path_to_feature_sets / 'covariates_below_5_durable.csv').covariate.values
full_set_durable_verifiable_covariates = pd.read_csv(
    path_to_feature_sets / 'full_set_durable_verifiable_covariates.csv'
).covariate.values
mosaiks_nearest_features = pd.read_csv(
    path_to_feature_sets / 'mosaiks_nearest_features.csv'
).covariate.values

non_linear_lambdas = np.concatenate((np.linspace(0, .15, 20, endpoint=False) ,np.linspace(.15, .5, 20)))
non_linear_lambdas.sort()

### Gap targeting

In [3]:
run_gap_targeting_grid(
    features=selected_durable_verifiable_covariates,
    country='malawi',
    save=SAVE_PATH / 'gap',
    lambdas=np.linspace(0, .5, 20),
    ns_quantiles=[10],
    hidden_layer_sizes=[64],
    extra_run_labels={'features': 'selected_durable'}
)

Fitting conditional program - QR method via nonparametric regression...


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 300/300 [00:01<00:00, 224.92it/s, val loss=1.57e-5]


Fitting conditional program - QR method via nonparametric regression...


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 300/300 [00:01<00:00, 247.97it/s, val loss=0.0342]


Fitting conditional program - QR method via nonparametric regression...


...

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 300/300 [00:01<00:00, 212.85it/s, val loss=0.0639]


Fitting conditional program - QR method via nonparametric regression...


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 300/300 [00:01<00:00, 248.05it/s, val loss=0.0524]


Fitting conditional program - QR method via nonparametric regression...


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 300/300 [00:00<00:00, 307.16it/s, val loss=0.0342]


In [11]:
run_gap_targeting_grid(
    features=selected_durable_verifiable_covariates[:4],
    country='malawi',
    save=SAVE_PATH / 'gap',
    lambdas=np.linspace(0, .5, 20),
    ns_quantiles=[10],
    hidden_layer_sizes=[64],
    extra_run_labels={'features': '4_selected_durable'}
)

Fitting conditional program - QR method via nonparametric regression...


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 300/300 [00:01<00:00, 282.83it/s, val loss=0]


Fitting conditional program - QR method via nonparametric regression...


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 300/300 [00:00<00:00, 316.56it/s, val loss=0.0329]


Fitting conditional program - QR method via nonparametric regression...


...

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 300/300 [00:00<00:00, 367.85it/s, val loss=0.0666]


Fitting conditional program - QR method via nonparametric regression...


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 300/300 [00:00<00:00, 360.32it/s, val loss=0.0539]


Fitting conditional program - QR method via nonparametric regression...


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 300/300 [00:00<00:00, 339.02it/s, val loss=0.0339]


In [12]:
run_gap_targeting_grid(
    features=mosaiks_nearest_features,
    country='malawi',
    save=SAVE_PATH / 'gap',
    lambdas=np.linspace(0, .5, 20),
    ns_quantiles=[10],
    hidden_layer_sizes=[64],
    extra_run_labels={'features': 'mosaiks'}
)

Fitting conditional program - QR method via nonparametric regression...


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 300/300 [00:04<00:00, 65.35it/s, val loss=1.31e-5]


Fitting conditional program - QR method via nonparametric regression...


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 300/300 [00:04<00:00, 62.38it/s, val loss=0.0468]


Fitting conditional program - QR method via nonparametric regression...


...

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 300/300 [00:04<00:00, 65.29it/s, val loss=0.111]


Fitting conditional program - QR method via nonparametric regression...


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 300/300 [00:04<00:00, 60.57it/s, val loss=0.0858]


Fitting conditional program - QR method via nonparametric regression...


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 300/300 [00:05<00:00, 57.75it/s, val loss=0.0549]


In [13]:
run_gap_targeting_grid(
    features=['ea_id'],
    country='malawi',
    save=SAVE_PATH / 'gap',
    lambdas=np.linspace(0, .5, 20),
    ns_quantiles=[10],
    hidden_layer_sizes=[64],
    extra_run_labels={'features': 'EA'}
)

Fitting conditional program - QR method via nonparametric regression...


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 300/300 [00:01<00:00, 171.38it/s, val loss=0.00267]


Fitting conditional program - QR method via nonparametric regression...


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 300/300 [00:01<00:00, 181.72it/s, val loss=0.0503]


Fitting conditional program - QR method via nonparametric regression...


...

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 300/300 [00:01<00:00, 186.32it/s, val loss=0.0996]


Fitting conditional program - QR method via nonparametric regression...


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 300/300 [00:01<00:00, 183.96it/s, val loss=0.0831]


Fitting conditional program - QR method via nonparametric regression...


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 300/300 [00:01<00:00, 183.00it/s, val loss=0.0568]


In [14]:
run_gap_targeting_grid(
    features=['district'],
    country='malawi',
    save=SAVE_PATH / 'gap',
    lambdas=np.linspace(0, .5, 20),
    ns_quantiles=[10],
    hidden_layer_sizes=[64],
    extra_run_labels={'features': 'district'}
)

Fitting conditional program - QR method via nonparametric regression...


100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 300/300 [00:01<00:00, 238.52it/s, val loss=0.000125]


Fitting conditional program - QR method via nonparametric regression...


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 300/300 [00:00<00:00, 308.51it/s, val loss=0.0435]


Fitting conditional program - QR method via nonparametric regression...


...

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 300/300 [00:01<00:00, 229.87it/s, val loss=0.0974]


Fitting conditional program - QR method via nonparametric regression...


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 300/300 [00:01<00:00, 230.86it/s, val loss=0.0813]


Fitting conditional program - QR method via nonparametric regression...


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 300/300 [00:01<00:00, 255.02it/s, val loss=0.0538]


### Parameter search

In [ ]:
run_gap_targeting_grid(
    features=full_set_durable_verifiable_covariates,
    country='malawi',
    save=SAVE_PATH / 'gap_parameter_search',
    lambdas=non_linear_lambdas,
    ns_quantiles=[5,10,20,50],
    extra_run_labels={'features': 'all_durable'}
)

In [ ]:
run_gap_targeting_grid(
    features=full_set_durable_verifiable_covariates,
    country='malawi',
    save=SAVE_PATH / 'gap_parameter_search',
    lambdas=non_linear_lambdas,
    ns_quantiles=[5, 20],
    hidden_layer_sizes=[4096],
    extra_run_labels={'features': 'all_durable'}
)

In [ ]:
run_gap_targeting_grid(
    features=mosaiks_nearest_features,
    country='malawi',
    save=SAVE_PATH / 'gap_parameter_search',
    lambdas=non_linear_lambdas,
    ns_quantiles=[5, 20],
    hidden_layer_sizes=[64, 128, 256, 512],
    extra_run_labels={'features': 'mosaiks'}
)

In [47]:
run_gap_targeting_grid(
    features=mosaiks_nearest_features,
    country='malawi',
    save=SAVE_PATH / 'gap_parameter_search',
    lambdas=non_linear_lambdas,
    ns_quantiles=[5],
    hidden_layer_sizes=[4096],
    extra_run_labels={'features': 'mosaiks'}
)

Fitting conditional program - QR method via nonparametric regression...


100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 300/300 [00:27<00:00, 10.78it/s, val loss=0]


Fitting conditional program - QR method via nonparametric regression...


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 300/300 [00:26<00:00, 11.33it/s, val loss=0.0731]


Fitting conditional program - QR method via nonparametric regression...


100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 300/300 [00:26<00:00, 11.46it/s, val loss=0.106]


Fitting conditional program - QR method via nonparametric regression...


100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 300/300 [00:26<00:00, 11.29it/s, val loss=0.109]


Fitting conditional program - QR method via nonparametric regression...


100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 300/300 [00:26<00:00, 11.25it/s, val loss=0.086]


### Rate targeting for comparison

In [6]:
# By looking at results I see that the post-transfer rate is in the range
# 0 - 0.38. So I'll run rate-targeting for that range to compare cost-benefit
"""
run_simulation(
    country="malawi",
    constraint="unconditional",
    method="qr",
    condtol=None,  # iterable
    uncondtol=None,  # iterable
    save="malawi_results.csv",
    features=None,
    extra_run_labels=None,
    district=None
)
"""
unconditional_tolerances = np.linspace(0.01, 0.4, 20)
run_simulation(
    country='malawi',
    constraint='unconditional',
    uncondtol=unconditional_tolerances,
    save=SAVE_PATH / 'rate_for_gap_comparison',
    features=full_country_covariates
)

Fitting conditional densities vs glm spline method...


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████| 300/300 [00:05<00:00, 52.51it/s, loss=-0.158, val_loss=-0.118]


Final Theta: tensor([[-0.0319,  0.7812,  0.0599,  ...,  0.1040, -0.2889,  0.1443],
        [ 0.3256, -0.2117,  0.3347,  ..., -0.7968,  0.3841,  0.7716],
        [ 1.1478, -0.2352, -0.5780,  ..., -0.0935, -0.1887,  0.1572],
        ...,
        [-0.5920, -0.4375,  0.0295,  ..., -0.0708, -0.1191,  0.1747],
        [-0.0688,  0.0512,  0.8323,  ...,  0.8371, -0.3488,  0.2082],
        [-0.6935,  0.1320, -0.7672,  ..., -0.5061,  0.3029, -0.3506]],
       dtype=torch.float64)
Alpha range: 0.0017594375784975643, 1.7594375784975642


100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 200/200 [00:57<00:00,  3.45it/s]


Alpha range: 0.0017594375784975643, 1.7594375784975642


...

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 200/200 [01:02<00:00,  3.20it/s]


Alpha range: 0.0017594375784975643, 1.7594375784975642


100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 200/200 [01:02<00:00,  3.19it/s]


Alpha range: 0.0017594375784975643, 1.7594375784975642


 16%|███████████████████████▎                                                                                                                     | 33/200 [00:21<01:00,  2.78it/s]IOPub message rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_msg_rate_limit`.

Current values:
ServerApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
ServerApp.rate_limit_window=3.0 (secs)



### Checking quantile regression

In [52]:
import seaborn as sns
sns.set_theme()

X, y, r, _ = load_malawi_data(full_set_durable_verifiable_covariates)

(X_train, y_train, r_train), (X_test, y_test, r_test) = split_data(
    X=X, y=y, r=r, p=0.6, random_seed=123
)

tt = GapTargetedTransfers(c_bar=2.15)

test_quantile_rates = []
train_quantile_rates = []
for hidden_layer_size in [64, 512]:
    for n_quantiles in [5,10, 20]:
        
        (quantiles, quantile_regressors, dataset) = tt.fit(
            X_train, y_train,  n_quantiles=n_quantiles, hidden_layer_size=hidden_layer_size
        )
        
        test_quantile_outcomes = []
        train_quantile_outcomes = []
        quantiles_to_try = np.linspace(0, .4, 25)
        for quantile in quantiles_to_try:
        
            estimated_quantile_wealth_levels = tt.get_baseline_wealth_at_quantile(X_test, quantile)
            test_quantile_outcomes.append(y_test.flatten() < estimated_quantile_wealth_levels.flatten())

            estimated_quantile_wealth_levels = tt.get_baseline_wealth_at_quantile(X_train, quantile)
            train_quantile_outcomes.append(y_train.flatten() < estimated_quantile_wealth_levels.flatten())
        
        test_quantile_outcomes = np.array(test_quantile_outcomes)
        train_quantile_outcomes = np.array(train_quantile_outcomes)
        
        test_quantile_indicators = pd.DataFrame(columns=quantiles_to_try, data=test_quantile_outcomes.transpose()).astype(int)
        test_quantile_rates_this_run = test_quantile_indicators.mean().to_frame()
        test_quantile_rates_this_run.reset_index(inplace=True)
        test_quantile_rates_this_run.columns=['quantile', 'actual_below_estimated']
        test_quantile_rates_this_run['n_quantiles'] = n_quantiles
        test_quantile_rates_this_run['hidden_layer_size'] = hidden_layer_size
        test_quantile_rates.append(test_quantile_rates_this_run)

        train_quantile_indicators = pd.DataFrame(columns=quantiles_to_try, data=train_quantile_outcomes.transpose()).astype(int)
        train_quantile_rates_this_run = train_quantile_indicators.mean().to_frame()
        train_quantile_rates_this_run.reset_index(inplace=True)
        train_quantile_rates_this_run.columns=['quantile', 'actual_below_estimated']
        train_quantile_rates_this_run['n_quantiles'] = n_quantiles
        train_quantile_rates_this_run['hidden_layer_size'] = hidden_layer_size
        train_quantile_rates.append(train_quantile_rates_this_run)

test_quantile_rates = pd.concat(test_quantile_rates)
train_quantile_rates = pd.concat(train_quantile_rates)

Fitting conditional program - QR method via nonparametric regression...


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 300/300 [00:01<00:00, 185.28it/s, val loss=0]


Fitting conditional program - QR method via nonparametric regression...


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 300/300 [00:01<00:00, 195.85it/s, val loss=0.0557]


Fitting conditional program - QR method via nonparametric regression...


...

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 300/300 [00:01<00:00, 191.42it/s, val loss=0.0675]


Fitting conditional program - QR method via nonparametric regression...


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 300/300 [00:01<00:00, 191.57it/s, val loss=0.0556]


Fitting conditional program - QR method via nonparametric regression...


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 300/300 [00:01<00:00, 190.61it/s, val loss=0.0313]


In [53]:
test_quantile_rates['dataset'] = 'test'
train_quantile_rates['dataset'] = 'train'

quantile_rates = pd.concat((test_quantile_rates, train_quantile_rates))

In [55]:
to_plot = quantile_rates[quantile_rates.dataset=='test']
fg = sns.relplot(
    data=to_plot, x='quantile', y='actual_below_estimated', hue='n_quantiles', 
    style='hidden_layer_size', kind='line',
    legend=True, aspect=12/8, height=8
)
for ax in fg.fig.axes:
    
    sns.lineplot(
        data=to_plot,
        x='quantile', y='quantile',
        color='red',
        linestyle='--',
        label='x=y',
        ax=ax,
        legend=True
    )
fg.set(xlabel='Quantile', ylabel='Fraction below estimated quantile wealth (OOS)')


NameError: name 'relplot' is not defined

### Clear runs from results

In [41]:
results_path = SAVE_PATH / 'gap_parameter_search' / 'malawi.csv'
results = pd.read_csv(results_path)

In [40]:
results[results.features != 'mosaiks'].to_csv(results_path, index=False)